# Software to record video from camera

Version: 1.0

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from IPython.display import display, clear_output, update_display, Image
from ipywidgets import Layout, Label
import ipywidgets as widgets
from datetime import date
from os.path import exists

from scipy.signal import argrelextrema

import time

import functions

import live
import global_

from ipywidgets import Output

# Function that returns a list of max and min points of a array


def max_min_pts(lst):

    i = 0
    test = argrelextrema(lst, np.less_equal, order=10)

    min_list = test[0].tolist()
    max_list = []
    min_list.pop(0)

    while True:
        if i + 2 > len(min_list):
            break

        if (min_list[i+1] - min_list[i] < 4):
            min_list.pop(i)
        else:
            i = i + 1

    for i in range(len(min_list)):

        max_area = -1

        if i + 1 < len(min_list):
            temp = lst[min_list[i]:min_list[i+1]]

            for j in range(len(temp)):
                if temp[j] > max_area:
                    max_area = temp[j]
                    area_index = min_list[i] + j

            max_list.append(area_index)

    return max_list, min_list


# Configure widgets
menu_path = widgets.Text(
    value='D:/Documentos/Unicamp/IC/Videos/Rec',
    placeholder='Type something',
    description='Save videos at:',
    disabled=False,
    layout=Layout(width='40%')
)

accordion = widgets.Accordion(children=[global_.live_logs])
accordion.set_title(0,'Logs')

outt10 = Output()

menu = widgets.VBox([menu_path,accordion,outt10])

display(menu)

resize = False
today = str(date.today())
cam_index = 1
width = 320
height = 240
size = 0
font = cv.FONT_HERSHEY_SIMPLEX

cam = live.cam
cap = cam.open_instance(cam,1,320,240)

ellipse = live.ellipse
contour = live.contour()
data = live.DataAnalysis()


def scale_contour(cnt, scale):
    M = cv.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = cnt - [cx,0]
    #cnt_scaled = cnt_norm * scale
    
    cnt_scaled = np.copy(cnt_norm)

    for i in range(len(cnt_norm)):
        cnt_scaled[i][0][0] = cnt_norm[i][0][0]*scale
    
    cnt_scaled = cnt_scaled + [cx,0]
    cnt_scaled = cnt_scaled.astype(np.int32)

    return cnt_scaled

def filters(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # image to grayscale
    blur = cv.medianBlur(gray,9)    # Apply median blur with kernel = 9
    gauss = cv.GaussianBlur(blur,(9,9),sigmaX=0,sigmaY=0)   # Apply Gaussian blur with kernel = 9
        
    return gauss  

video_getter = live.VideoGet(1).start()



number = 0
times = np.array([])



while(True): 

    start_time = time.time()
    
    #ret, frame = cap.read() # Read image from specified camera

    frame = video_getter.frame

    #start_time = time.time()

    cam.current_frame = frame
    
    
    # Record raw frame obtained from camera, if camera is recording
    if cam.recording:
        writer.write(frame)
        # Indicates that the video is being recorded
        cv.circle(frame,(25,25), 15, (0,0,255), -1)
        cv.putText(frame,'REC',(50,40), font, 1.3,(0,0,255),2,cv.LINE_AA)

    elif cam.adjustment:
        # Apply filters on both frames
        filtered = filters(frame)

        _ , binary_img = cv.threshold(filtered,cam.thold,255,cv.THRESH_BINARY)

        # Detect binary edges
        edges = cv.Canny(binary_img, 50, 100, 5, L2gradient= True)

        # Find contours
        contours,_ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
        
        # Detect greatest contour of both frames and get its index
        max_area = -1
        for i, c in enumerate(contours):
            if cv.arcLength(c,True) > max_area: 
                max_area = cv.arcLength(c,True)
                max_i = i
        
        # Apply scale to main contour
        scaled_contour = scale_contour(contours[max_i],cam.scale/100)

        # Draw contours on both frames
        cv.drawContours(frame,scaled_contour,-1,(255,0,255),1)

        # Obtain ellipse param as  (center x,center y), (height,width), ângle
        center,dim,angle = cv.fitEllipseDirect(scaled_contour)

        # Draw ellipse 
        cv.ellipse(frame, (center,dim,angle), (255,0,255), 1)     


        cv.line(frame,(cam.ver_axis,0),(cam.ver_axis,cam.height),(255,0,255),1)



        left_x = np.array([])
        right_x = np.array([])

        for i in range (len(scaled_contour)):
            if abs(scaled_contour[i][0][1] - cam.hor_axis) <= 5:
                if scaled_contour[i][0][0] <= cam.ver_axis:
                    left_x = np.append(left_x,scaled_contour[i][0][0])
                else:
                    right_x = np.append(right_x,scaled_contour[i][0][0])

        if left_x.size > 0 and right_x.size > 0:        
            cv.circle(frame,(int(np.average(left_x)),cam.hor_axis), 3, (0,0,255), -1)
            cv.circle(frame,(int(np.average(right_x)),cam.hor_axis), 3, (0,0,255), -1)
        
        cv.line(frame,(0,cam.hor_axis),(cam.width,cam.hor_axis),(255,0,255),1)

    if cam.analysis:

        # Apply filters and threshold to obtain a binary image
        filtered = functions.filters(frame) 
        _ , binary_img = cv.threshold(filtered,cam.thold,255,cv.THRESH_BINARY)

        # Fit ellipse 
        contours,rectangle,drawing = ellipse.fit(ellipse,binary_img)

        # Keep information on contour area
        data.area_record = np.append(data.area_record, 
                                        cv.contourArea(contours))

        data.time_record = np.append(data.time_record, 
                                        time.time() - first_time)
        
        data.new_data = True
        data.start()


        cv.imshow('Representation',drawing)

        cv.ellipse(frame, ellipse.param, (90,90,255), 1)    # Draw ellipse on main frame


    
    cv.imshow("Camera", frame)

    video_getter.new_frame = False
       
    # Reads user input
    k = cv.waitKey(1) & 0xff

    # Test if user pressed "Spacebar" and if so, start/stop camera recording
    if k == ord(" "):

        # Test if camera was recording and stop recording
        if cam.recording:
            cam.stop_rec(cam,writer) 
        # Else, start recording frames
        else:
            writer = cam.start_rec(cam,menu_path.value)          

    # Test if user pressed a number (0,1,...,9) from keyboard and switch 
    # camera currently displayed accordingly
    elif (k >= 48 and k <= 57) and (not cam.recording) and (not cam.adjustment):
        cap.release()
        cv.destroyAllWindows()
        cam_index = k - 48
        cap = cam.open_instance(cam_index,width,height)

    # Test if user pressed "Tab" on keyboard and changes camera resolution
    # from (320x240) to (640x240) and so on
    elif (k == 9) and (not cam.recording) and (not cam.adjustment):
        cap.release()
        cv.destroyAllWindows()
        if size == 0:
            width = 640
            height = 480
            size = 1
        elif size == 1:
            width = 320
            height = 240
            size = 0
        cap = cam.open_instance(cam_index,width,height)

    # Test id user pressed "Enter" on keyboard, and start to adjust main
    # parameters necessary 
    elif (k == 13):
        if cam.adjustment:
            cam.analysis = True
            cam.adjustment = False
            cv.destroyWindow('Comandos')
            first_time = time.time()
            
        else: 
            cam.get_filter_parameters(cam)

    # Test if user pressed "Esc" on keyboard and exit program
    elif k == 27:
        if cam.recording:
            writer.release()
            cv.setWindowTitle("Camera","Camera")
            with global_.live_logs:
                print("Video {0} was saved successfuly".format(video))
        cap.release()
        cv.destroyAllWindows()
        break

    # Loop that waits for new Thread to read another frame from cam
    while True:
        if video_getter.new_frame:
            break
    
    #if number == 30:
    #    print(time.time() - start_time)

    with outt10:
        number = number + 1
        times = np.append(times,time.time() - start_time)
        


        if int(np.sum(times)) == 1:
            clear_output(True)
            print(np.sum(times))
            print("Reading %s Frames per Second" % (number))

            if data.area_record.size > 0:
                print(data.freq)
                #print(data.time_record)
            times = np.array([])
            number = 0   
    

